In [1]:
import os
import pandas as pd
import jieba
import numpy as np
from collections import Counter
import pickle
from tqdm import tqdm

D:\Anaconda\envs\dl\lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
train_df=pd.read_csv('../data/processed/train.csv')
test_df=pd.read_csv('../data/processed/test.csv')
val_df=pd.read_csv('../data/processed/val.csv')
print(f'训练集{train_df.shape}')
print(f'测试集{test_df.shape}')
print(f'验证集{val_df.shape}')
print(f"\n类别: {train_df['label'].unique()}")

训练集(45462, 4)
测试集(9743, 4)
验证集(9742, 4)

类别: [13  9  6  7  3  0 10  2  5 12  4  1 11  8]


In [3]:
# 中文停用词表
stopwords = set([
    '的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都',
    '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会',
    '着', '没有', '看', '好', '自己', '这', '中', '以', '来', '个',
    '地', '为', '他', '得', '她', '对', '么', '里', '后', '能', '再',
    '而', '被', '从', '把', '让', '与', '等', '别', '之', '这个',
])

print(f"停用词数量: {len(stopwords)}")

停用词数量: 53


In [4]:
def tokenize(text):
    words=jieba.cut(text)
    words=[w for w in words if w not in stopwords]
    return words
# 测试分词
sample_text = train_df.iloc[0]['text']
print("原文:")
print(sample_text[:100])
print("\n分词后:")
print(" / ".join(tokenize(sample_text)[:20]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ysn\AppData\Local\Temp\jieba.cache


原文:
成长型基金猜想2009：幸福岁月能否延续
　　实习记者 杨颖桦 本报记者 张桔 
　　究竟是全市场基金还是成长型基金会在2009年走牛？种种迹象显示答案是后者。 
　　银河证券基金研究中心的统计显示，

分词后:


Loading model cost 1.215 seconds.
Prefix dict has been built successfully.


成长型 / 基金 / 猜想 / 2009 / ： / 幸福 / 岁月 / 能否 / 延续 / 
 / 　 / 　 / 实习 / 记者 /   / 杨颖桦 /   / 本报记者 /   / 张桔


In [5]:
# 对所有数据分词
from tqdm import tqdm

# 重要！先启用pandas的进度条
tqdm.pandas()
train_df['words']=train_df['text'].progress_apply(tokenize)
test_df['words']=test_df['text'].progress_apply(tokenize)
val_df['words']=val_df['text'].progress_apply(tokenize)

print("\n 分词完成！")

# 查看分词后的结果
print("\n示例（前3条）:")
for i in range(3):
    print(f"\n第{i+1}条:")
    print(f"标签: {train_df.iloc[i]['label']}")
    print(f"分词: {' '.join(train_df.iloc[i]['words'][:15])}...")

100%|██████████| 9742/9742 [00:32<00:00, 303.73it/s]


 分词完成！

示例（前3条）:

第1条:
标签: 13
分词: 成长型 基金 猜想 2009 ： 幸福 岁月 能否 延续 
 　 　 实习 记者  ...

第2条:
标签: 9
分词: 《 侠盗 车手 》 将 推出 续作   明年 亮相 E3 
 　 　 由...

第3条:
标签: 6
分词: 组图 ： 6 款 性感 丝袜 最配 夏季 超短 
 　 　 导读 ： 穿...


In [6]:
all_words=[]
for word in train_df['words']:
    all_words.extend(word)
    
word_freq=Counter(all_words)
print(f"总词: {len(all_words)}")
print(f"不重复词数:{word_freq}")

min_freq=5
word_freq={word:freq for word, freq in word_freq.items() if freq>=min_freq}
print(f"保留词数: {len(word_freq)}")

sorted_words=sorted(word_freq.items(), key=lambda x:x[1], reverse=True)
# 构建词到ID的映射
vocab = {'<PAD>': 0, '<UNK>': 1}  # 特殊token
for word, freq in sorted_words:
    vocab[word] = len(vocab)

print(f"\n词表大小: {len(vocab):,}")

print(f"\nTop 20 高频词:")
for i, (word, freq) in enumerate(sorted_words[:20], 1):
    print(f"{i:2d}. {word}: {freq:,}")

# 保存词表
with open('../data/processed/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

print("\n 词表已保存到 data/processed/vocab.pkl")

总词: 23111713


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




 词表已保存到 data/processed/vocab.pkl


In [7]:
# Cell 7: 文本转ID序列
print("将文本转换为ID序列...")

def words_to_ids(words, vocab, max_len=512):
    """将词列表转换为ID列表"""
    ids = [vocab.get(word, vocab['<UNK>']) for word in words]

    if len(ids) > max_len:
        ids = ids[:max_len]
    else:
        ids = ids + [vocab['<PAD>']] * (max_len - len(ids))

    return ids

# 转换
train_df['ids'] = train_df['words'].apply(lambda x: words_to_ids(x, vocab))
val_df['ids'] = val_df['words'].apply(lambda x: words_to_ids(x, vocab))
test_df['ids'] = test_df['words'].apply(lambda x: words_to_ids(x, vocab))

print("ID转换完成！")

# 示例
idx = 0
print(f"\n示例:")
print(f"原文: {train_df.iloc[idx]['text'][:50]}...")
print(f"分词: {' '.join(train_df.iloc[idx]['words'][:10])}...")
print(f"ID序列前20个: {train_df.iloc[idx]['ids'][:20]}")

将文本转换为ID序列...
ID转换完成！

示例:
原文: 成长型基金猜想2009：幸福岁月能否延续
　　实习记者 杨颖桦 本报记者 张桔 
　　究竟是全市场基...
分词: 成长型 基金 猜想 2009 ： 幸福 岁月 能否 延续 
...
ID序列前20个: [29558, 34, 9530, 151, 10, 1720, 5850, 1432, 1596, 5, 3, 3, 2944, 45, 6, 1, 6, 1160, 6, 89093]


In [8]:
# Cell 8: 保存标签映射
import pickle

# THUCNews 14个类别
label_map_save = {
    0: '体育', 1: '娱乐', 2: '家居', 3: '彩票', 4: '房产', 5: '教育',
    6: '时尚', 7: '时政', 8: '星座', 9: '游戏', 10: '社会', 11: '科技',
    12: '股票', 13: '财经'
}

with open('../data/processed/label_map.pkl', 'wb') as f:
    pickle.dump(label_map_save, f)

print("标签映射已保存")
print("\n标签映射:")
for k, v in label_map_save.items():
    print(f"{k:2d} → {v}")

标签映射已保存

标签映射:
 0 → 体育
 1 → 娱乐
 2 → 家居
 3 → 彩票
 4 → 房产
 5 → 教育
 6 → 时尚
 7 → 时政
 8 → 星座
 9 → 游戏
10 → 社会
11 → 科技
12 → 股票
13 → 财经


In [9]:
# Cell 9: 保存最终数据并总结
import pickle

print("保存最终处理结果...")

# 保存完整数据（包含text, label, label_name, words, ids等）
train_df.to_pickle('../data/processed/train_processed.pkl')
val_df.to_pickle('../data/processed/val_processed.pkl')
test_df.to_pickle('../data/processed/test_processed.pkl')

print("\n" + "="*60)
print("🎉 数据预处理全部完成！")
print("="*60)

print("\n数据统计:")
print(f"  训练集: {len(train_df):,} 条")
print(f"  验证集: {len(val_df):,} 条")
print(f"  测试集: {len(test_df):,} 条 （有label，可以评估）")
print(f"  词表大小: {len(vocab):,}")
print(f"  类别数: 14")
print(f"  序列长度: 512")


保存最终处理结果...

🎉 数据预处理全部完成！

数据统计:
  训练集: 45,462 条
  验证集: 9,742 条
  测试集: 9,743 条 （有label，可以评估）
  词表大小: 117,355
  类别数: 14
  序列长度: 512
